Load data with Pandas from a CSV

In [2]:
import pandas as pd
df = pd.read_csv(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv", sep='|', low_memory=False)
print(f"Number of rows in file: {df.shape[0]}")

C:\Users\es422\AppData\Local\Temp\ipykernel_10492\2124138203.py:2: DtypeWarning: Columns (10,13,17,19,21,24,25,27,30,38,43,53,54,55,57,58,70,73,74,76,77,78,79,80,84,89,92,93,94,95,96,97,100,101,112,115,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv", sep='|')


Number of rows in file: 6187847


Load data with Pandas from a Parquet

In [3]:
import pandas as pd

df = pd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")
print(f"Number of rows in file: {df.shape[0]}")

Number of rows in file: 6187847


Test PySpark from parquet

In [4]:
import os
import sys

# Set Hadoop environment variables
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = os.environ['PATH'] + r';C:\hadoop\bin'

# Create SparkSession with datetime rebase configuration
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("LocalSpark") \
    .config("spark.driver.memory", "16g") \
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "LEGACY") \
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY") \
    .config("spark.sql.parquet.int96RebaseModeInWrite", "LEGACY") \
    .getOrCreate()

df = spark.read.parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")
print(f"Number of rows in file: {df.count()}")

Number of rows in file: 6187847


Dask from CSV

In [12]:
import dask.dataframe as dd

df = dd.read_csv(
    r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv",
    sep='|',
    dtype = 'object',
    low_memory=False,  
)
print(f"Number of rows: {df.shape[0].compute()}")

Number of rows: 6187847


Dask from parquet

In [10]:
import dask.dataframe as dd

df = dd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")
print(f"Number of rows: {df.shape[0].compute()}")

Number of rows: 6187847
